In [ ]:
import os
import math
import random
import glob

import warnings
warnings.filterwarnings("ignore", category=Warning)

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import numpy as np
import scipy
from scipy import stats

import pandas as pd
import xlsxwriter
import joblib

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, f1_score

import xgboost as xgb
import shap

In [ ]:
## for binarizing sex variable
def getTF(x):
    if x=='M': return 0.0
    elif x=='F': return 1.0
    else: raise

In [ ]:
df_cli = pd.read_excel('/workspace/list_of_patients_with_clinical_information.xlsx', sheet_name=0)

df_cli = df_cli[['PET_ID', 'Sex', 'Symptom_Onset_Age', 'Symptom_Onset_Age_60', 'Gap_in_3months', 'HY_rnd', 
                 'RT_RA', 'RT_LA', 'RT_RL', 'RT_LL', 'FT_R', 'FT_L', 'LA_R', 'LA_L', 'RG_RA', 'RG_LA',
                 '1_VS/Calc_Bi-1', '2_PP/Calc_Bi-1', '3_PC/Calc_Bi-1', '4_AC/Calc_Bi-1', '5_AP/Calc_Bi-1', 
                 '1_Lim/Calc_Bi-1', '2_Exe/Calc_Bi-1', '3_Sen/Calc_Bi-1', 'Whole_putamen', 'LID_within_5Y', 'LEDD', 'Cumulative_LED']]

df_with_na = df_cli[df_cli.isna().any(axis=1)]

target_list = ['HY_rnd', 'RT_RA', 'RT_LA', 'RT_RL', 'RT_LL', 'FT_R', 'FT_L', 'LA_R', 'LA_L', 'RG_RA', 'RG_LA']

## impute missing values with the median value
for var in target_list:
    df_cli[var] = df_cli[var].fillna(df_cli[var].median())

df_cli_filled_na = df_cli.iloc[df_with_na.index]
df_cli_filled_na = df_cli_filled_na[target_list] 


df_cli['Tremor_total'] = df_cli['RT_RA'] + df_cli['RT_LA'] + df_cli['RT_RL'] + df_cli['RT_LL']
df_cli['Brady_total'] = df_cli['FT_R'] + df_cli['FT_L'] + df_cli['LA_R'] + df_cli['LA_L']
df_cli['Rigid_total'] = df_cli['RG_RA'] + df_cli['RG_LA']

df_cli['RT_UEx'] = df_cli['RT_RA'] + df_cli['RT_LA']
df_cli['RT_LEx'] = df_cli['RT_RL'] + df_cli['RT_LL']

df_cli['FT'] = df_cli['FT_R'] + df_cli['FT_L']
df_cli['LA'] = df_cli['LA_R'] + df_cli['LA_L']

df_cli['RG_UEx'] = df_cli['RG_RA'] + df_cli['RG_LA']


cont_list = ['Symptom_Onset_Age', 'Symptom_Onset_Age_60', 'Gap_in_3months', 'HY_rnd', 'Tremor_total', 'Brady_total', 'Rigid_total',
             '1_VS/Calc_Bi-1', '2_PP/Calc_Bi-1', '3_PC/Calc_Bi-1', '4_AC/Calc_Bi-1', '5_AP/Calc_Bi-1', 
             '1_Lim/Calc_Bi-1', '2_Exe/Calc_Bi-1', '3_Sen/Calc_Bi-1', 'Whole_putamen', 'LEDD', 'Cumulative_LED',
             'RT_UEx', 'RT_LEx', 'FT', 'LA', 'RG_UEx']

up_list = ['RT_RA', 'RT_LA', 'RT_RL', 'RT_LL', 'FT_R', 'FT_L', 'LA_R', 'LA_L', 'RG_RA', 'RG_LA']
cat_list = ['Sex']
id_list = ['PET_ID']

df_cli = df_cli.copy()

## sex to either 0 or 1
df_cli.loc[:,'Sex_Z'] = df_cli["Sex"].map(lambda x:getTF(x))

## updrs 0~4 --> 0~2
for var in up_list:
    new_col = var+'_Z'
    # df_cli_338.loc[:, new_col] = df_cli_338[var].map(lambda x:x/4)
    df_cli.loc[:, new_col] = df_cli[var].map(lambda x:x/2)
    
scaler = MinMaxScaler(feature_range=(0,2))
for var in cont_list:
    new_col = var+'_Z'
    df_cli.loc[:, new_col] = scaler.fit_transform(df_cli[[var]])


## drop the original columns and keep the normalized columns
df_z = df_cli.drop(['Sex', 'Symptom_Onset_Age', 'Gap_in_3months', 'HY_rnd', 'RT_RA', 'RT_LA', 'RT_RL', 'RT_LL', 'FT_R', 'FT_L', 'LA_R', 'LA_L', 'RG_RA', 'RG_LA', 
                    'Symptom_Onset_Age_60', 'Tremor_total', 'Brady_total', 'Rigid_total', 'RT_UEx', 'RT_LEx', 'FT', 'LA', 'RG_UEx',
                    '1_VS/Calc_Bi-1', '2_PP/Calc_Bi-1', '3_PC/Calc_Bi-1', '4_AC/Calc_Bi-1', '5_AP/Calc_Bi-1', 
                    '1_Lim/Calc_Bi-1', '2_Exe/Calc_Bi-1', '3_Sen/Calc_Bi-1', 'Whole_putamen', 'LEDD', 'Cumulative_LED'], axis=1)
display(df_z)

## Machine learning models: LR, RF, XGBoost

## Variables: SNBRs only, SNBR + clinical variables

### - utils for evaluation

In [ ]:
def calculate_accuracy(y_truth, y_pred, class_names, title='Confusion matrix', fold_num=0, SAVE_FOLDER='/workspace/folder_to_save_results/'):
    num_TN = 0
    num_FP = 0
    num_TP = 0
    num_FN = 0
    
    for idx in range(0, len(y_truth)):
        if y_truth[idx] == 0:
            if y_pred[idx] == 0:
                num_TN += 1
            else:
                num_FP += 1
        else:
            if y_pred[idx] == 1:
                num_TP += 1
            else:
                num_FN += 1
    print ('       TN =', num_TN, '/ FP =', num_FP)
    print ('       FN =', num_FN, '/ TP =', num_TP)
    
    sen = float(num_TP) / float(num_TP + num_FN)
    print ('       sensitivity:', round(sen*100, 2), '%')
    
    spec = float(num_TN) / float(num_TN + num_FP)
    print ('       specificity:', round(spec*100, 2), '%')
    
    test_acc = accuracy_score(y_truth, y_pred)
    print ('       Accuracy >> ' + str(round(test_acc * 100, 2)) + '%\n')
    
    plot_confusion_matrix(confusion_matrix(y_truth, y_pred), classes=class_names, title=title, fold_num=fold_num, SAVE_FOLDER=SAVE_FOLDER)
    
    return sen, spec

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues, fold_num=0, SAVE_FOLDER='/workspace/codes/folder_to_save_results/'):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    
    if normalize:
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix')


    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize='x-large')
    plt.colorbar(pad=0.02)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, fontsize='large')
    plt.yticks(tick_marks, classes, fontsize='large', rotation=90)

   
    thresh = cm.max() / 2.
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize and cm_normalized is not None:
            text = f"{cm[i, j]}\n({cm_normalized[i, j]:.1f}%)"
        else:
            text = f"{cm[i, j]}"
        
        plt.text(j, i, text,
                 horizontalalignment="center",
                 verticalalignment="center",
                 color="white" if cm[i, j] > thresh else "black", size='x-large')

    
    plt.tight_layout()
    plt.ylabel('True label', fontsize='x-large')
    plt.yticks(tick_marks, ['LID-', 'LID+'], va='center', fontsize='x-large')
    plt.xlabel('Predicted label', fontsize='x-large')

    plt.savefig(SAVE_FOLDER + f'/Confusion_matrix_{title}.png', dpi=300, bbox_inches='tight') 
    plt.show()

In [ ]:
def plot_auroc_3(tpr, fpr, data, currrent_ml, SAVE_FOLDER):

    plt.figure(figsize=(6,6), dpi=150)
    
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1.5, alpha=0.8, color='b', linestyle='solid', label='%s, AUC=%0.3f' % (data, round(roc_auc,3)))

    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='grey', alpha=.8)

   
    plt.xlim([-0.01, 1.01])
    plt.ylim([-0.01, 1.01])
    plt.xlabel('1 - Specificity', fontsize = 12)
    plt.ylabel('Sensitivity', fontsize = 12)
    plt.title(currrent_ml, fontsize = 14)
    plt.legend(loc="lower right", fontsize = 12)
    
    plt.gcf().patch.set_alpha(1.0)
    plt.savefig(SAVE_FOLDER + f'/ROC_{currrent_ml}.png', dpi=300, bbox_inches='tight') 
    plt.show()

### - LR, RF, and XGBoost functions

-  (df) X_train: row is for the subjects, column is for the variables
-  (df) y_train: 0 or 1
-  (df) X_test
-  (df) y_test

In [ ]:
def logistic_regression_run_2(X_train, y_train, X_test, y_test, skf, my_model, SAVE_FOLDER):
    
    if not os.path.exists(SAVE_FOLDER):
        os.makedirs(SAVE_FOLDER)
        
 
    n_features = []
    fpr_list, tpr_list, thr_list, data_list = [], [], [], []
    sen, spec, sen_roc, spec_roc, accuracy, f1_scores, aurocs = [], [], [], [], [], [], []
    
    y_prob_test_all_folds = []
    
    cur_fold = 0

    X_test = np.asarray(X_test, dtype="float32")
    y_test = np.asarray(y_test)

    
    ######## LR ##############################################
    print ("[logistic regression]")
    clsf_LR = LogisticRegression()
    
    print ("grid search ......")
    grid = {"class_weight": ["balanced", None], "C": np.logspace(-3, 3, 7), "penalty": ["l1","l2","elasticnet"], "class_weight": ["balanced"]}
    gs = GridSearchCV(clsf_LR, grid, scoring='roc_auc', cv=skf)

    print ("fit ......")
    gs = gs.fit(X_train, np.ravel(y_train, order='C'))
    print ("Hyperparameter: ", gs.best_params_)
    
    clsf_LR = LogisticRegression(class_weight = gs.best_params_['class_weight'], C = gs.best_params_['C'], penalty = gs.best_params_['penalty'])
    clsf_LR = clsf_LR.fit(X_train, np.ravel(y_train, order='C'))


    ######### Calculate evaluation metrics ###########################
    y_prob_test_set = clsf_LR.predict_proba(X_test)[:,clsf_LR.classes_[1]]    ## for the test set

    fpr, tpr, th = roc_curve(y_test, y_prob_test_set)
    
    youden = np.argmax(tpr-fpr)
    print ("Youden index:", th[youden])
   
    final_predictions = (y_prob_test_set >= th[youden]).astype(bool) 
    
    print ("Label: ", y_test)
    print ("Prediction: ", final_predictions)
    print ("Threshold: ", th[youden])
    
    _accuracy = accuracy_score(y_test, final_predictions)
    print ('accuracy: ', str(_accuracy*100) + '%')
    
    _f1_score = f1_score(y_test, final_predictions)
    print ('f1 score: ', _f1_score)
    
    _sen, _spec = calculate_accuracy(y_test, final_predictions, ["LID-", "LID+"], f'{my_model}_LR_test', 0, SAVE_FOLDER)
    print ('Sensitivity: ', _sen)
    print ('Specificity: ', _spec)

    _auroc = auc(fpr, tpr)
    
    #================================================================================

    plot_auroc_3(tpr, fpr, 'Test set', f'{my_model}_LR', SAVE_FOLDER)

    df_test_results = pd.DataFrame({'Model': [f'({my_model})_LR'], 'Fold': ['test'], 
                            'Accuracy': _accuracy, 'Sensitivity': _sen, 'Specificity': _spec, 'F1 score': _f1_score, 'AUROC': _auroc})


    return y_test, y_prob_test_set, df_test_results, clsf_LR

In [ ]:
def rf_run_2(X_train, y_train, X_test, y_test, skf, my_model, SAVE_FOLDER, seed):
    
    if not os.path.exists(SAVE_FOLDER):
        os.makedirs(SAVE_FOLDER)
        
 
    n_features = []
    fpr_list, tpr_list, thr_list, data_list = [], [], [], []
    sen, spec, sen_roc, spec_roc, accuracy, f1_scores, aurocs = [], [], [], [], [], [], []
    
    y_prob_test_all_folds = []
    
    cur_fold = 0

    X_test = np.asarray(X_test, dtype="float32")
    y_test = np.asarray(y_test)

    
    ######## LR ##############################################
    print ("[random forest]")
    clsf_RF = RandomForestClassifier(random_state=seed)

    print ("grid search ......")
    grid = { 'random_state': [999], 'class_weight': ['balanced'], 'criterion': ['gini'], 'n_estimators': [100, 200, 300],
        'min_samples_split': [2, 4, 8], 'max_features': [1, 3], 'max_depth': [2, 4, 6] }
    gs = GridSearchCV(clsf_RF, grid, scoring='roc_auc', cv=skf)
    

    print ("fit ......")
    gs = gs.fit(X_train, np.ravel(y_train, order='C'))
    print ("Hyperparameter: ", gs.best_params_)

    clsf_RF = RandomForestClassifier(random_state = 999, class_weight = gs.best_params_['class_weight'], criterion = gs.best_params_['criterion'], 
                                     n_estimators = gs.best_params_['n_estimators'], min_samples_split = gs.best_params_['min_samples_split'],
                                     max_features = gs.best_params_['max_features'], max_depth = gs.best_params_['max_depth'])
    clsf_RF = clsf_RF.fit(X_train, np.ravel(y_train, order='C'))


    ######### Calculate evaluation metrics ###########################
    y_prob_test_set = clsf_RF.predict_proba(X_test)[:,clsf_RF.classes_[1]]    ## for the test set

    fpr, tpr, th = roc_curve(y_test, y_prob_test_set)
    
    youden = np.argmax(tpr-fpr)
    print ("Youden index:", th[youden])
   
    final_predictions = (y_prob_test_set >= th[youden]).astype(bool) 
    
    print ("Label: ", y_test)
    print ("Prediction: ", final_predictions)
    print ("Threshold: ", th[youden])
    
    _accuracy = accuracy_score(y_test, final_predictions)
    print ('accuracy: ', str(_accuracy*100) + '%')
    
    _f1_score = f1_score(y_test, final_predictions)
    print ('f1 score: ', _f1_score)
    
    _sen, _spec = calculate_accuracy(y_test, final_predictions, ["LID-", "LID+"], f'{my_model}_RF_test', 0, SAVE_FOLDER)
    print ('Sensitivity: ', _sen)
    print ('Specificity: ', _spec)

    _auroc = auc(fpr, tpr)
    
    #================================================================================

    plot_auroc_3(tpr, fpr, 'Test set', f'{my_model}_RF', SAVE_FOLDER)

    df_test_results = pd.DataFrame({'Model': [f'({my_model})_RF'], 'Fold': ['test'], 
                            'Accuracy': _accuracy, 'Sensitivity': _sen, 'Specificity': _spec, 'F1 score': _f1_score, 'AUROC': _auroc})


    return y_test, y_prob_test_set, df_test_results, clsf_RF

In [ ]:
def xgboost_run_2(X_train, y_train, X_test, y_test, skf, my_model, SAVE_FOLDER, seed):
    
    if not os.path.exists(SAVE_FOLDER):
        os.makedirs(SAVE_FOLDER)
        
 
    n_features = []
    fpr_list, tpr_list, thr_list, data_list = [], [], [], []
    sen, spec, sen_roc, spec_roc, accuracy, f1_scores, aurocs = [], [], [], [], [], [], []
    
    y_prob_test_all_folds = []
    
    cur_fold = 0

    X_test = np.asarray(X_test, dtype="float32")
    y_test = np.asarray(y_test)

    
    ######## LR ##############################################
    print ("[XGBoost]")
    clsf_xgb = xgb.XGBClassifier(verbosity = 0, random_state=seed)

    print ("grid search ......")
    grid = { 'random_state': [999], 'objective':['binary:logistic'], 'max_depth': [1, 3],
        'subsample': [0.4, 0.6, 0.8], 'colsample_bytree': [0.1, 0.3, 0.5], 'n_estimators': [300, 500, 700, 1000] }

    gs = GridSearchCV(clsf_xgb, grid, scoring='roc_auc', cv=skf)
    
    print ("fit ......")
    gs = gs.fit(X_train, np.ravel(y_train, order='C'))
    print ("Hyperparameter: ", gs.best_params_)

    clsf_xgb = xgb.XGBClassifier(random_state = 999, objective = 'binary:logistic', max_depth = gs.best_params_['max_depth'], 
                                 subsample = gs.best_params_['subsample'], colsample_bytree = gs.best_params_['colsample_bytree'],
                                 n_estimators = gs.best_params_['n_estimators'])
    clsf_xgb = clsf_xgb.fit(X_train, np.ravel(y_train, order='C'))


    ######### Calculate evaluation metrics ###########################
    y_prob_test_set = clsf_xgb.predict_proba(X_test)[:,clsf_xgb.classes_[1]]    ## for the test set

    fpr, tpr, th = roc_curve(y_test, y_prob_test_set)
    
    youden = np.argmax(tpr-fpr)
    print ("Youden index:", th[youden])
   
    final_predictions = (y_prob_test_set >= th[youden]).astype(bool) 
    
    print ("Label: ", y_test)
    print ("Prediction: ", final_predictions)
    print ("Threshold: ", th[youden])
    
    _accuracy = accuracy_score(y_test, final_predictions)
    print ('accuracy: ', str(_accuracy*100) + '%')
    
    _f1_score = f1_score(y_test, final_predictions)
    print ('f1 score: ', _f1_score)
    
    _sen, _spec = calculate_accuracy(y_test, final_predictions, ["LID-", "LID+"], f'{my_model}_XGB_test', 0, SAVE_FOLDER)
    print ('Sensitivity: ', _sen)
    print ('Specificity: ', _spec)

    _auroc = auc(fpr, tpr)
    
    #================================================================================

    plot_auroc_3(tpr, fpr, 'Test set', f'{my_model}_XGB', SAVE_FOLDER)

    df_test_results = pd.DataFrame({'Model': [f'({my_model})_XGB'], 'Fold': ['test'], 
                            'Accuracy': _accuracy, 'Sensitivity': _sen, 'Specificity': _spec, 'F1 score': _f1_score, 'AUROC': _auroc})


    return y_test, y_prob_test_set, df_test_results, clsf_xgb

### - SHAP function

In [ ]:
def plot_shap (X_test, shap_values, my_model, ml_method, SAVE_FOLDER, current_seed):      ## X_train, y_train are dataframes

    ## beeswarm plot
    shap.summary_plot(shap_values, X_test, show=False, max_display=10)
    plt.xlabel("SHAP value")
    plt.title(f'{my_model} {ml_method} (Test set)', fontsize=14)
    plt.savefig(SAVE_FOLDER + f'/Shapley {my_model} {ml_method} s{current_seed} bee.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # bar plot
    plt.figure()
    shap.summary_plot(shap_values, X_test, plot_type="bar", show=False, max_display=10) 
    plt.xlabel("mean(|SHAP value|)")
    plt.title(f'{my_model} {ml_method} (Test set)', fontsize=14)
    for bar in plt.gca().patches:
        bar.set_facecolor("red")
    plt.savefig(SAVE_FOLDER + f'/Shapley {my_model} {ml_method} s{current_seed} bar.png', dpi=300, bbox_inches='tight')
    plt.close()

### - Code for running the ML models

In [ ]:
## need to use the same seeds used to split the data for training the deep learning model (from notebook: preparation_for_cross_validation.ipynb)
random_seed = [768, 187, 488, 758, 915]
fold_list = ['Fold_0', 'Fold_1', 'Fold_2', 'Fold_3', 'Fold_4']

target_path = '/workspace/data_folder'
SAVE_FOLDER = '/workspace/save_results_folder'

model_list = ['snbr', 'snbr + clinical_vars']

df_finale_results = pd.DataFrame()

for my_model in model_list:
    
    for current_fold, current_seed in zip(fold_list, random_seed):

        current_folder = os.path.join(target_path, current_fold)
        
        ## choose the right columns for each model
        df_current = df_z
        
        if my_model=='snbr':
            df_current = df_current[['PET_ID', 'LID_within_5Y', '1_VS/Calc_Bi-1_Z', '2_PP/Calc_Bi-1_Z', '3_PC/Calc_Bi-1_Z', '4_AC/Calc_Bi-1_Z', '5_AP/Calc_Bi-1_Z', 
                                     '1_Lim/Calc_Bi-1_Z', '2_Exe/Calc_Bi-1_Z', '3_Sen/Calc_Bi-1_Z']]
            
        elif my_model=='snbr + clinical_vars':
            df_current = df_current[['PET_ID', 'LID_within_5Y', 'Sex_Z', 'Symptom_Onset_Age_Z', 'Gap_in_3months_Z',
                                     'HY_rnd_Z', 'RT_UEx_Z', 'RT_LEx_Z', 'FT_Z', 'LA_Z', 'RG_UEx_Z',
                                     '1_VS/Calc_Bi-1_Z', '2_PP/Calc_Bi-1_Z', '3_PC/Calc_Bi-1_Z', '4_AC/Calc_Bi-1_Z', '5_AP/Calc_Bi-1_Z']]      
        else:
            print('Wrong model name!')
            raise

        
        ## X_train, y_train, X_test, y_test
        train_list = glob.glob(current_folder+'/train/*_img.nii.gz')
        train_list = [os.path.basename(x).split('_')[0] + '_' + os.path.basename(x).split('_')[1] for x in train_list]

        test_list = glob.glob(current_folder+'/test/*_img.nii.gz')
        test_list = [os.path.basename(x).split('_')[0] + '_' + os.path.basename(x).split('_')[1] for x in test_list]

        X_train = df_current[df_current['PET_ID'].isin(train_list)].drop(['PET_ID', 'LID_within_5Y'], axis=1)
        y_train = df_current[df_current['PET_ID'].isin(train_list)]['LID_within_5Y']

        X_test = df_current[df_current['PET_ID'].isin(test_list)].drop(['PET_ID', 'LID_within_5Y'], axis=1)
        y_test = df_current[df_current['PET_ID'].isin(test_list)]['LID_within_5Y']


        original_path = os.path.join(target_path, current_fold, 'train')
        total_list = sorted(os.listdir(original_path))
        total_list = total_list[0::2]

        X = [s[:-11] for s in total_list]
        Y = []

        for i in total_list:
            if '_no_' in i: 
                Y.append(0)
            elif '_yes_' in i: 
                Y.append(1)
            else:
                print('I am a wanderer!')

        skf = StratifiedKFold(n_splits=5, random_state=current_seed, shuffle=True)

        ## logistic regression
        y_test_LR, y_prob_test_set_LR, df_test_results_LR, clsf_LR = logistic_regression_run_2(X_train, y_train, X_test, y_test, skf, f'{my_model}_s{current_seed}', SAVE_FOLDER)
        y_test_RF, y_prob_test_set_RF, df_test_results_RF, clsf_RF = rf_run_2(X_train, y_train, X_test, y_test, skf, f'{my_model}_s{current_seed}', SAVE_FOLDER, current_seed)
        y_test_XGB, y_prob_test_set_XGB, df_test_results_XGB, clsf_XGB = xgboost_run_2(X_train, y_train, X_test, y_test, skf, f'{my_model}_s{current_seed}', SAVE_FOLDER, current_seed)
        
        df_finale_results = pd.concat([df_finale_results, df_test_results_LR, df_test_results_RF, df_test_results_XGB], axis=0)
        
        
        # Save models
        model_path_LR = os.path.join(SAVE_FOLDER, f's{current_seed}_LR_model.joblib')
        model_path_RF = os.path.join(SAVE_FOLDER, f's{current_seed}_RF_model.joblib')
        model_path_XGB = os.path.join(SAVE_FOLDER, f's{current_seed}_XGB_model.joblib')

        joblib.dump(clsf_LR, model_path_LR)
        joblib.dump(clsf_RF, model_path_RF)
        joblib.dump(clsf_XGB, model_path_XGB)
        
        
        ## SHAP
        ## 1. LR
        explainer = shap.Explainer(clsf_LR, X_train)
        shap_values = explainer(X_test)
        plot_shap (X_test, shap_values, my_model, 'LR', SAVE_FOLDER, current_seed)
        
        ## 2. RF
        explainer = shap.TreeExplainer(clsf_RF, X_train)
        shap_values = explainer(X_test, check_additivity=False)[:,:,1]
        plot_shap (X_test, shap_values, my_model, 'RF', SAVE_FOLDER, current_seed)
    
        ## 3. XGBoost
        explainer = shap.TreeExplainer(clsf_XGB, X_train)
        shap_values = explainer(X_test, check_additivity=False)
        plot_shap (X_test, shap_values, my_model, 'XGBoost', SAVE_FOLDER, current_seed)
        

        
display(df_finale_results)
df_finale_results.to_excel('/workspace/save_results_folder/total_result.xlsx', index=False)

## Checking correlation coefficient between features

### Pearson correlation coefficient for continuous variables --> SNBRs

In [ ]:
var_list = ['1_VS/Calc_Bi-1_Z', '2_PP/Calc_Bi-1_Z', '3_PC/Calc_Bi-1_Z', '4_AC/Calc_Bi-1_Z', '5_AP/Calc_Bi-1_Z',
            '1_Lim/Calc_Bi-1_Z', '2_Exe/Calc_Bi-1_Z', '3_Sen/Calc_Bi-1_Z']

results = []

for var1, var2 in itertools.combinations(var_list, 2):
    corr, p_value = stats.pearsonr(df_z[var1], df_z[var2])
    results.append((var1, var2, corr, p_value))

corr_df = pd.DataFrame(results, columns=['Variable 1', 'Variable 2', 'Pearson Correlation', 'P-Value'])
corr_df = corr_df.sort_values(by=['Pearson Correlation'], ascending=False)

display(corr_df)    

### Spearman rank correlation coefficient for clinical variables

In [ ]:
var_list = ['Sex_Z', 'RT_RA_Z', 'RT_LA_Z', 'RT_RL_Z', 'RT_LL_Z', 'FT_R_Z', 'FT_L_Z', 'LA_R_Z', 'LA_L_Z', 'RG_RA_Z', 'RG_LA_Z',
            'Symptom_Onset_Age_Z', 'Symptom_Onset_Age_60_Z', 'Gap_in_3months_Z',
            'HY_rnd_Z', 'Tremor_total_Z', 'Brady_total_Z', 'Rigid_total_Z', 
            '1_VS/Calc_Bi-1_Z', '2_PP/Calc_Bi-1_Z', '3_PC/Calc_Bi-1_Z', '4_AC/Calc_Bi-1_Z', '5_AP/Calc_Bi-1_Z']

results = []

for var1, var2 in itertools.combinations(var_list, 2):
    corr, p_value = stats.spearmanr(df_z[var1], df_z[var2])
    results.append((var1, var2, corr, p_value))

corr_df = pd.DataFrame(results, columns=['Variable 1', 'Variable 2', 'Spearman Correlation', 'P-Value'])
corr_df = corr_df.sort_values(by=['Spearman Correlation'], ascending=False)

display(corr_df)
corr_df.to_excel('/workspace/save_results_folder/corr_df.xlsx', index=False)